In [9]:
# Importing numpy and functions
import numpy as np
import helpers as h
from implementations import *

# Loading the data
X, xHeader, Y, yHeader, indexedX, indexedXheader, indexedY, indexedYheader = loadTrainingData()
print('')
# Cleaning/feature engineering the data
yClean, xClean, xHeaderClean, removedFeatures = dataCleaning(Y,X,xHeader)
print('')
# Making a balanced data set to force the model to not just predict negatively all the time
yBalanced, xBalanced, balancePrior = balanceData(yClean,xClean)
print('')
# Adding dummy variables and replacing the remaining invalid values by the mean
tx = makeTrainingData(xBalanced)
print(f'The resultant dataarray tx has shape {tx.shape}')

Data successfully loaded, there are 321 features and 328135 samples, the shapes of the unindexed data is:
y: (328135, 1), x: (328135, 321)

For a threshold of 0.7, there are 144 good features, and 177 bad features
There remains in the data 289870 samples with at most 5 missing values
The number of invalid entries remaing in the dataset is 272938
That is 0.006538802834987332 parts of the whole dataset
Removed 21050 samples with outliers more than 10 standard deviations from the mean. There remains 268820 samples in the dataset.
Standardized data by subtracting the mean and dividing by the standard deviation

Created a balanced subset of the data, with 46448 samples, 23224 each of positive and negative samples

Added dummy variable and replaced invalid entries with zeros
The resultant dataarray tx has shapes (46448, 145)


In [10]:
# Initializing the parameters at zero and setting some constants
initial_w = np.zeros(tx.shape[1])
K = 5
gamma = 0.01
max_iter = 1000

# Training a model with gradient descent with momentum
w_gd_m, loss_gd_m = k_fold_cross_validation(yBalanced,tx,K,initial_w,max_iter,gamma, mse_gd_momentum, compute_loss)
print('')
# Training a model with logistic
w_logistic, loss_logistic = k_fold_cross_validation(yBalanced,tx,K,initial_w,max_iter,gamma)


Run 1 yielded a loss improvement from 0.5012917115177611 to 0.1552742181694449
Run 2 yielded a loss improvement from 0.5071044133476856 to 0.15513740607157933
Run 3 yielded a loss improvement from 0.5058127018299247 to 0.15725589352782068
Run 4 yielded a loss improvement from 0.49542469587684357 to 0.15300931120396857
Run 5 yielded a loss improvement from 0.4903649477877059 to 0.15504066222900187
-----------------------------------------------------------------------------------------
Averaging the parameters, the loss improves from 0.5 to 0.1565452697989322

Run 1 yielded a loss improvement from 0.6931471805599084 to 0.47357670299947235
Run 2 yielded a loss improvement from 0.6931471805599084 to 0.47152853137194933
Run 3 yielded a loss improvement from 0.6931471805599084 to 0.4760067175154304
Run 4 yielded a loss improvement from 0.6931471805599083 to 0.4749024434713034
Run 5 yielded a loss improvement from 0.6931471805599083 to 0.4825389674534961
-------------------------------------

In [3]:
# Loading the test data
xTest, xIndexedHeader = loadData('./Data/x_test.csv')
print(xTest.shape)

(109379, 322)


In [16]:
# Making predictions
pred_gd = makePredictions(w_logistic,xTest[:,1:],xHeader,xHeaderClean,balancePrior)
pred_logistic = makePredictions(w_logistic,xTest[:,1:],xHeader,xHeaderClean)
# Counting predicted positive cases
print(np.sum(pred_gd),np.sum(pred_logistic))


Added dummy variable and replaced invalid entries with zeros
Added dummy variable and replaced invalid entries with zeros
0.0 32500.0


In [5]:
pred_logistic[pred_logistic == 0] = -1
h.create_csv_submission(xTest[:,0],pred_logistic,'./Predictions/balancedDataNoPrior.csv')